# ADM-HW3 Group 6
## Nguyen Phu Hien | Omirbanu Nurassilova | Silvio Trampetti

### NOTED: We have attached a function file with our main ipynb file here on github. For more details about the function please check that file.

## Step 1: Data
### The data is downloaded from https://www.kaggle.com/PromptCloudHQ/airbnb-property-data-from-texas

## Step 2: Creating documents

Importing the necessary function files:

In [1]:
import os
os.chdir('C:\Master Sapienza\Algorithmic Methods in Data Mining\homework\hw3')

In [2]:
import csv
import pandas as pd
import nltk
import math
import pickle
import datetime
import numpy as np
import myfunction as mf # function file
import folium
from geopy import distance

We created tsv documents as follows:<br />
_ Open the csv file and skip the first line.<br />
_ Read each of the remaining lines and save it as a tsv file in a seperate folder (dataset).

In [ ]:
counter = 1
with open('Airbnb_Texas_Rentals.csv', 'r', encoding='utf-8' ) as csvfile:
    reader = csv.reader(csvfile)
    row1 = next(reader) # here we skip the first row of the .csv file
    for row in reader:
        if row: # if row is not empty, write a file with this row
            with open('C:\Master Sapienza\Algorithmic Methods in Data Mining\homework\hw3\dataset\doc_%s.txt'% str(counter),
                      'w', encoding='utf-8') as tsvfile_out:
                writer = csv.writer(tsvfile_out, delimiter='\t') # convert csv to tsv file
                writer.writerow(row)
                counter = counter + 1

## Step 3: Search Engine

### Preprocessing<br />


First we count the total number of documents available in our folder

In [3]:
# total number of documents
path, dirs, files = next(os.walk("dataset/"))
doc_count = len(files) #18259

Next step we preprocess the documents by:<br />
1. Removing stop words<br />
2. Removing puntuation<br />
3. Lemmatizing and removing the https<br />

Then we save the preprocessed file into a new folder (filteredset) for convenience.

In [ ]:
# removing stopwords
import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
#word_tokenize accepts a string as an input, not a file. 
stop_words = set(stopwords.words('english'))
#define a counter
for i in range(1,doc_count+1):
    with open('dataset/doc_%s.txt'%str(i), encoding='utf-8') as f:
        line = f.read()# Use this to read file content as a stream: 
        words = line.split()
        lemma = nltk.wordnet.WordNetLemmatizer() #lemmatization
        for r in words:
            if not r in stop_words:
                r = nltk.word_tokenize(r)
                r=[word.lower() for word in r if word.isalpha()]   
                r = ''.join(r) # convert r (list) back to string for lemmatization and concatenation
                if r != 'https': # remove https
                    r = lemma.lemmatize(r)
                    appendFile = open(r'filteredset/filtered_doc_%s.txt' % str(i),'a', encoding='utf-8') 
                    appendFile.write(" "+r) 
                    appendFile.close()

Next important step is to create a dictionary:

In [ ]:
dictionary = {}
files = os.listdir("filteredset/") #get all the files in filtered data set
for file in files:
    with open('filteredset/'+file, encoding='utf-8') as f:
        items = [i.strip() for i in f.read().split()]
    if items is not '':
        dictionary[file.replace(".txt", "")] = items
#create a dictionary with filename as key and words as value        

Then we try to modify the dictionary to the form of {word1: {filename1: [pos1, pos2]}{filename2: [pos1, pos2]}, ...}, ...}<br/>
Moving on we save this dictionary as a pkl for future usage

In [ ]:
dictionary = mf.make_indices(dictionary) # indexing our dictionary
dictionary = mf.fullIndex(dictionary) # reverse the index
#save the dictionary into pkl file using pickle
f = open("dictionary.pkl","wb")
pickle.dump(dictionary,f)
f.close()

We can load the dictionary into our ipynb file, for example as follows:

In [ ]:
with open('dictionary.pkl', 'rb') as f:
    data = pickle.load(f)

### 3.1) Conjunctive query

### Execute the query<br/>
Executing the conjunctive query as the first search. Here the input of the search is: "private room with beach view".

In [4]:
mf.exec_query_s_1(input())

private room with beach view


,description,city,url
title,,,
Room by the Bay with private balcony,"My place is within walking distance to parks, public transport, and Galveston Bay. NASA Space Center Houston is 9 miles away, Kemah Boardwalk 5, historic San Jacinto battleground and monument 7, Sylvan Beach 3, and 25 minutes east of downtown Houston . You’ll love my place because of the kitchen, the coziness, and the views. My place is good for couples, solo adventurers, and business travelers. As of June 2, we have completed refurbishing of our air conditioning for the comfort of our guests.",La Porte,https://www.airbnb.com/rooms/13092788?location=Beach%20City%2C%20TX
Beautiful 3BR Corpus Christi Condo,"With 3 bedrooms, 2 bathrooms and room for 8 guests, this condo provides wonderful Corpus Christi accommodations just 2 minutes from the beach. Pool access, WiFi and a private balcony with canal views create a spectacular backdrop for your stay.",Corpus Christi,https://www.airbnb.com/rooms/10598523?location=Baffin%20Bay%2C%20TX
Beach front Condo with Balcony,Our condo is across the street from the ocean. It has perfect views from the private balcony and is a 2 minute walk from the sand! It is light and refreshing in a wonderful location. The bathroom is separate and there is a futon for guests to stay in the living room. It is a 1 bedroom with a fully supplied kitchen and will sleep 4 adults.,Corpus Christi,https://www.airbnb.com/rooms/18483952?location=Aransas%20Pass%2C%20TX
Toes In The Water,"Built in 2013 a two story home featuring 4 bedrooms, 3 bathrooms, sleeps 20 and a great open concept living room big enough to accommodate all your friends and family and give them the majestic endless panoramic views of the Ocean. Two Master suites each with it's private bathroom and access to the 2nd floor balcony to enjoy those beautiful sunrises and sunsets while sipping on your favorite drink.",Surfside Beach,https://www.airbnb.com/rooms/17805292?location=Angleton%2C%20TX
Sapphire Penthouse #2901,"Located at one of South Padre Island’s most luxurious condominiums, Sapphire 2901, provides stunning balcony and window views of both the beach and the bay. The property offers an elegant dining table that can accommodate up to eight people. The Sapphire Condominiums offers two Olympic size swimming pools, a full service day spa, private movie theater, state of the art fitness center, a teen game room for all guest to enjoy.",South Padre Island,https://www.airbnb.com/rooms/18752099?location=Brownsville%2C%20TX
Beautiful Oceanfront Condo with Private Balcony,"A centrally-located vacation oceanfront home right on the Seawall, only one hundred steps from the beach!\n\nOpen space with lots of light, high ceilings, fully furnished. Wake up to stunning views sunrise on the beach anywhere you sleep. The master bedroom, loft bedroom, and living room all have views of the ocean front. Cook a delicious meal in our fully-equipped kitchen and eat together at the dining table while watching sunset on the beach. Dip into the hot tub at one of two pools.",Galveston,https://www.airbnb.com/rooms/18447939?location=Bayou%20Vista%2C%20TX
Beach front Condo with Balcony,Our condo is across the street from the ocean. It has perfect views from the private balcony and is a 2 minute walk from the sand! It is light and refreshing in a wonderful location. The bathroom is separate and there is a futon for guests to stay in the living room. It is a 1 bedroom with a fully supplied kitchen and will sleep 4 adults.,Corpus Christi,https://www.airbnb.com/rooms/18483952?location=Baffin%20Bay%2C%20TX
Private Room with Kitchen & Bathroom Adjacent,"Our place can accommodate 4. We are close to art and culture, the beach, restaurants and dining, and family-friendly activities. Houston is a melting pot of different cultures-you're sure to find places that suit your taste! \nYou’ll love our place because of the high ceilings, the coziness, the location, the views, and the kitchen(which you are free to use). Our h

Another search with the query of "nice and quiet place".

In [5]:
mf.exec_query_s_1(input())

nice and quiet place


,description,city,url
title,,,
Clean and safe place close to DFW Airport (Room 3),"Our house is in a very quiet, nice and safe neighborhood. The DFW airport is just 15 minutes drive away. The DART 401 Bus Line is 10 minutes walk away. Restaurants and Parks are within walking distance. You will access to our kitchen as well.",Irving,https://www.airbnb.com/rooms/17593615?location=Carrollton%2C%20TX
Plano Pad,"A nice guest room in a nice , quiet &amp; happy Asian home in Plano. Few steps from a beautifully golf court, walk able to shopping centers, restaurants, pubs. Nice big trees, large yard in a safe neighborhood, an affordable place to visit!",Plano,https://www.airbnb.com/rooms/7092779?location=Allen%2C%20TX
"Walkable Cherrywood Loft, close to UT and Downtown","Walking distance to about 2 dozen restaurants / bars / coffee shops, a few blocks away from the UT football stadium and campus, and about 2 miles from downtown... yet tucked away in the nice and quiet neighborhood of Cherrywood (aka French Place).\n\nThe loft is set back away back from the street with its own walkway and entrance and has a great mix of natural light and shade from the trees. It's newly remodeled and has a powerful AC for those hot Texas summers.\n\nCome check out The East Side!",Austin,https://www.airbnb.com/rooms/13941193?location=Colorado%20River%2C%20TX
Cozy Apartment with swimming pool,"You’ll love my place because of the coziness, the high ceilings, and the people. My place is good for couples, solo adventurers, business travelers, families (with kids), or if your in town for work , and furry friends (pets).Our neighborhood is very quiet and relaxing ,nice to take walks down by creeks or drive down to reservoir to fish,very spacious and open country like setting but also very close to town, very private and clean area.",Highlands,https://www.airbnb.com/rooms/16602404?location=Channelview%2C%20TX
Check our long term rates B,"My place is close to A&amp;M University, Blinn College. You’ll love my place because of new construction, safe neighborhood, clean, nice amenities. My place is good for couples, solo adventurers, and business travelers. The house is in a quiet neighborhood, it also includes amenities such as high speed wifi, and cable. Full access to common areas include the kitchen, living room and use of the washer and dryer. The house was built in 2013, and still feels new. Kitchen has all necessary tools to cook",Bryan,https://www.airbnb.com/rooms/15219946?location=Bryan%2C%20TX
Cozy Apartment with swimming pool,"You’ll love my place because of the coziness, the high ceilings, and the people. My place is good for couples, solo adventurers, business travelers, families (with kids), or if your in town for work , and furry friends (pets).Our neighborhood is very quiet and relaxing ,nice to take walks down by creeks or drive down to reservoir to fish,very spacious and open country like setting but also very close to town, very private and clean area.",Highlands,https://www.airbnb.com/rooms/16602404?location=Anahuac%2C%20TX
cozy apt near airports,"My place is close to both the DFW and Love Field airports. They're both about a 15 minute drive away (depending on the Dallas traffic, of course.) \n\nYou’ll love my place because of the ambiance, the outdoor space, the natural light, and the neighborhood. It's located across from the Las Colinas Country Club and the Four Seasons resort. It's nice, quiet, and just a 20 minute drive to downtown Dallas. My place is good for couples, solo adventurers, and business travelers.",Irving,https://www.airbnb.com/rooms/13256966?location=Coppell%2C%20TX
Completely Remodeled Home in West Plano,"My place is close to University of Texas Dallas, restaurants and dining, parks. Five minutes drive to George Bush and Central Expwy and Highway 75, which can take you to anywhere in Dallas. You’ll love the nice and quiet neighborhood, the comfy beds, and spacious high ceiling living area. Plus, everything is brand new and up

### 3.2) Conjunctive query & Ranking score

### Calculate TF-IDF score

As of theory:<br />
TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).<br />
IDF(t) = log_e(Total number of documents / Number of documents with term t in it).<br/>
The first step we try to make use of our former dictionary to create a new dictionary with term count:

In [ ]:
#input = {word: {filename: [pos1, pos2]}, ...}, ...}
#output = {word: {filename: term_count}, ...}, ...}
new_dict = {}
for k, v in data.items():
    sub_dict = {}
    for k1, v1 in v.items():
        term_count = len(v1) # term count in a document
        sub_dict.update({k1 : term_count})
    new_dict.update({k:sub_dict})
    del sub_dict # delete the old sub-dict for the term

In [ ]:
for k, v in new_dict.items():
    for k1, v1 in v.items():
        v[k1] = mf.tf_idf(v1, mf.total_count(k1), doc_count, mf.contain_count(k))
    new_dict[k] = v
#new_dict with tf-idf score        

We now have the new dictionary with the format of:

{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}), ...],
term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}), ...],
...}

Then we save the new_dict to a new pkl file for future usage.

In [ ]:
f = open("newdictionary.pkl","wb")
pickle.dump(new_dict,f)
f.close()

In [ ]:
with open('newdictionary.pkl', 'rb') as f:
    newdata = pickle.load(f)

### Execute the query<br/>
For this search we keep the top 10 documents with the highest scores by heap data structure. If the number of documents matched is less than 10, the search will return all the possible results.<br/>
For the first search, we use the same query as the previous search "private room with beach view" to observe the difference between the first two methods: 

In [6]:
mf.exec_tfidf_search(input())

private room with beach view


,description,city,url,similarity
title,,,,
Room by the Bay with private balcony,"My place is within walking distance to parks, public transport, and Galveston Bay. NASA Space Center Houston is 9 miles away, Kemah Boardwalk 5, historic San Jacinto battleground and monument 7, Sylvan Beach 3, and 25 minutes east of downtown Houston . You’ll love my place because of the kitchen, the coziness, and the views. My place is good for couples, solo adventurers, and business travelers. As of June 2, we have completed refurbishing of our air conditioning for the comfort of our guests.",La Porte,https://www.airbnb.com/rooms/13092788?location=Beach%20City%2C%20TX,0.97
Beautiful 3BR Corpus Christi Condo,"With 3 bedrooms, 2 bathrooms and room for 8 guests, this condo provides wonderful Corpus Christi accommodations just 2 minutes from the beach. Pool access, WiFi and a private balcony with canal views create a spectacular backdrop for your stay.",Corpus Christi,https://www.airbnb.com/rooms/10598523?location=Baffin%20Bay%2C%20TX,0.96
Private Room with Kitchen & Bathroom Adjacent,"Our place can accommodate 4. We are close to art and culture, the beach, restaurants and dining, and family-friendly activities. Houston is a melting pot of different cultures-you're sure to find places that suit your taste! \nYou’ll love our place because of the high ceilings, the coziness, the location, the views, and the kitchen(which you are free to use). Our home is good for couples, solo adventurers, business travelers, and families. Feel secure in our home. Our rooms have keyed entry.",Houston,https://www.airbnb.com/rooms/17040609?location=Channelview%2C%20TX,0.84
Just Breathe,"Just Breathe is 3600 sq ft of living space plus a ground level bar. This house has it all! Luxurious accommodations and decor, tvs in every room including a 60 inch in the bar, a playground for the kids, 3 living rooms to relax after a fun filled day at the beach, and more. Every room has a private bathroom plus an additional full bathroom for guest that may be on the sofa bed. There are decks on both levels with plenty of seating to enjoy the view. We would love to welcome your family!",Bolivar Peninsula,https://www.airbnb.com/rooms/17515618?location=Bolivar%20Peninsula%2C%20TX,0.84
Room by the Bay with private balcony,"My place is within walking distance to parks, public transport, and Galveston Bay. NASA Space Center Houston is 9 miles away, Kemah Boardwalk 5, historic San Jacinto battleground and monument 7, Sylvan Beach 3, and 25 minutes east of downtown Houston . You’ll love my place because of the kitchen, the coziness, and the views. My place is good for couples, solo adventurers, and business travelers. As of June 2, we have completed refurbishing of our air conditioning for the comfort of our guests.",La Porte,https://www.airbnb.com/rooms/13092788?location=Clear%20Lake%20Shores%2C%20TX,0.84
Sapphire Penthouse #2901,"Located at one of South Padre Island’s most luxurious condominiums, Sapphire 2901, provides stunning balcony and window views of both the beach and the bay. The property offers an elegant dining table that can accommodate up to eight people. The Sapphire Condominiums offers two Olympic size swimming pools, a full service day spa, private movie theater, state of the art fitness center, a teen game room for all guest to enjoy.",South Padre Island,https://www.airbnb.com/rooms/18752099?location=Brownsville%2C%20TX,0.83
Beautiful Oceanfront Condo with Private Balcony,"A centrally-located vacation oceanfront home right on the Seawall, only one hundred steps from the beach!\n\nOpen space with lots of light, high ceilings, fully furnished. Wake up to stunning views sunrise on the beach anywhere you sleep. The master bedroom, loft bedroom, and living room all have views of the ocean front. Cook a delicious meal in our fully-equipped kitchen and eat together at the dining table while watching sunset on the beach. Dip into the hot tub at one of two pools.",Galveston,https://ww

Secondly we try some random search as:"I love a place for sleeping"

In [7]:
mf.exec_tfidf_search(input())

I love a place for sleeping


,description,city,url,similarity
title,,,,
Peaceful Lakefront Retreat,"My place is spacious and has a view of the lake from every room. You’ll love the outdoors space, the unique architecture, and the relaxing, quiet atmosphere. It's perfect for all.\nWe have sleeping spaces for 5. Also a very large living room to accommodate air mattresses, etc. Only $10/person beyond 5. \nI do require that bookings are made by persons age 25 or over.\nThe beach at Little Elm Park is absolutely the best in the Dallas-Ft. Worth Metroplex, and is only 5 minutes away from my house.",Little Elm,https://www.airbnb.com/rooms/13751175?location=Celina%2C%20TX,0.86
Carriage House in Bastrop,"Bastrop Carriage House is close to nightlife, the airport, family-friendly activities and thirty minutes to the Formula One, Austin's ACL and SXSW annual events. You’ll love my place because of the coziness, the location, and the people; the carriage house is good for couples, solo adventurers, business travelers, and families (with kids). In the historic district surrounded by architecture and fabulous restaurants, great for walking. 1 Bed, 2nd sleeping loft. So 2 beds",Bastrop,https://www.airbnb.com/rooms/16947229?location=Bastrop%20County%2C%20TX,0.45
Sleeping Porch Awesome View,The Sleeping Porch is perched on a elevated two acre hill over looking the hill country along what was once the Old Spanish Trail. A unique experience for those looking for authentic hill country old ranch style experience.. A place for those who love privacy and seclusion amount the sights and sounds of nature. The Sleeping Porch is screened in awesome panoramic view with a nice southerly breeze yet heavy drapes can be closed to allow the AC to cool down the space in a matter of minutes.,Boerne,https://www.airbnb.com/rooms/13408497?location=Boerne%2C%20TX,0.42
Superbowl Ready Large Loft,"My place is on IH-45 south. It located close to the University of Houston, Downtown, and Midtown and only 10 minutes from NRG stadium. . You’ll love my place because of It is 2800 Square feet with 2 big bedrooms and large common area with a full bar, 16 foot big screen tv and sound system. Perfect for a large group and extra space for sleeping. The place is secluded and safe. . My place is good for couples, business travelers, families (with kids), and big groups.",Houston,https://www.airbnb.com/rooms/16630196?location=Channelview%2C%20TX,0.42
Sleeping Porch Awesome View,The Sleeping Porch is perched on a elevated two acre hill over looking the hill country along what was once the Old Spanish Trail. A unique experience for those looking for authentic hill country old ranch style experience.. A place for those who love privacy and seclusion amount the sights and sounds of nature. The Sleeping Porch is screened in awesome panoramic view with a nice southerly breeze yet heavy drapes can be closed to allow the AC to cool down the space in a matter of minutes.,Boerne,https://www.airbnb.com/rooms/13408497?location=Center%20Point%2C%20TX,0.42
Carriage House in Bastrop,"Bastrop Carriage House is close to nightlife, the airport, family-friendly activities and thirty minutes to the Formula One, Austin's ACL and SXSW annual events. You’ll love my place because of the coziness, the location, and the people; the carriage house is good for couples, solo adventurers, business travelers, and families (with kids). In the historic district surrounded by architecture and fabulous restaurants, great for walking. 1 Bed, 2nd sleeping loft. So 2 beds",Bastrop,https://www.airbnb.com/rooms/16947229?location=Bastrop%2C%20TX,0.42
Carriage House in Bastrop,"Bastrop Carriage House is close to nightlife, the airport, family-friendly activities and thirty minutes to the Formula One, Austin's ACL and SXSW annual events. You’ll love my place because of the coziness, the location, and the people; the carriage house is good for couples, solo adventurers, business travelers, and families (with kids). In the historic district surrounded b

## Step 4: Define a new score!

*Some additional columns are created and attached to the dataframe in order to score them in this step.*<br/>
_First of all, we decide to grade them by the time when they were posted. For instance, if the date of posting of the advertisement is later, the place receives a higher grade.<br/>
_The second criterion is the room rate. We calculate average per city, and then compare each rate with its
average. If it is less than the average by more than 20% we award the most point for the rating, and if it is less than the average of within 20% limit (of average) we give some credits. And if it is more expensive than average by more than 20% we deduct some points.<br/>
_The third criterion is related to the location of the place which is demonstrated in the 'city column'. We refer to the information from the website: https://livability.com/tx/real-estate/best-places-to-live-in-texas for grading criteria. There are ten cities situated in Texas and according to this these ten cities are the best in Texas to live. So we added it as a criterion.<br/>
_The fourth criterion is the number of bedrooms available. In case the query contains the following words: house,
home, apartment we award some marks since they indicate that the place has many bedrooms.

The search give the top 10 output with the ranking available. The top 10 is kept by heap data structure. If there are fewer than 10 matched, we return all the possible results. For instance of the first search query "small house in city center":

In [8]:
mf.exec_query_s_2(input())

small house in city center


,title,description,city,url
Ranking,,,,
1,Private Rm/Bath @ the Gold House!,"This unique private guest room at the Gold House offers the amenities of home in a city full of things to do! The entire downstairs living room, kitchen, and dining area is open to guests. Come on down to SA town and experience Texan hospitality for yourself!\n\nPlease note: there is a second guest room at the Gold house, so if this listing is booked, do check with me to see if the second room is available! Thanks for your interest!\n\n\nCute House:\nIt's a private room upstairs with a full bed, small desk, walk-in closet, Ironing Board, and en suite bathroom. The window has a nice view of the back yard and wooded easement. The back yard is large with a little garden in the back. The kitchen was recently renovated with nice granite counter tops and new appliances. Also, there is a brand new running/hiking trail one mile away. Parking is safe and free in the form of my driveway.\n\nMore than Two People? No problem:\nThere is a second guestroom in the loft with a full bed that can be reserved via my other listing. http://airbnb.com/rooms/826725\n\nBreakfast/Snacks:\nThe living room/dining room/patio/kitchen are all fair game for guests. Breakfast is provided by way of oatmeal and any items found on the counter (Hot tea and coffee too). \n\nFour-Legged Friends: \nI do have two wonderful dogs (lab and husky), but don't worry they aren't allowed upstairs. They stay outside during the day, and sleep in their beds downstairs at night. They are both full of sugar - but if you have a fear of dogs then this place is not for you(URL HIDDEN)\nLocation:\nIt is literally right next to a brand new grocery store but you wouldn't know it from the back yard. It is 10-15 minutes from La Cantera - the gorgeous outdoor shopping mall. As well as Six Flags Fiesta Texas and UTSA. It is 5-10 minutes from the medical center area where all the hospitals are. It is 3 miles from IH-10, which is the main highway to get to downtown where the Riverwalk is (about 20 minutes by car). Airport - about 15-20 minutes away.\n\n\nThings to look up for your trip regardless of whether you choose the Gold House for your stay (we want you to have fun!):\n\nIn San Antonio...\n- The Riverwalk \n- Market Square\n- The Spanish Missions (Includes the Alamo of course...you can rent bikes to explore all of them!)\n- Six Flags Fiesta Texas Theme Park\n- Sea World Theme Park\n- Morgan's Wonderland (Special Needs Theme Park)\n- San Antonio Botanical Gardens\n- San Antonio Zoo\n\nWithin an Hour of San Antonio...\n- Schlitterbahn Water Park (Amazing rides, Bring Food, and Free Parking)\n- Natural Bridge Caverns (wondrous caves to stay cool)\n- Natural Bridge Wildlife Ranch (African Safari Texas Style)\n- Gruene, Texas (adorable hill country town with shopping and tubing on the river)\n\n\nThe entire downstairs!\n\nSafe neighborhood - within walking distance to a grocery store!\n\nBus stop is about a 10 minute walk from the front door.\n\nCheckout is more flexible if there are no guests arriving the same night you're leaving.",San Antonio,https://www.airbnb.com/rooms/1284796?location=Brazos%20River%2C%20TX
2,"Walk to Knox St., Highland Park Private room/bath","My place is close to art and culture, restaurants and dining, parks, and the city center. You’ll love my place because of the ambiance, the neighborhood, and the comfy bed. My place is good for couples, solo adventurers, and business travelers. There is a small dog in the house.",Dallas,https://www.airbnb.com/rooms/17400806?location=Coppell%2C%20TX
3,Private Rm/Bath @ the Gold House!,"This unique private guest room at the Gold House offers the amenities of home in a city full of things to do! The entire downstairs living room, kitchen, and dining area is open to guests. Come on down to SA town and experience Texan hospitality for yourself!\n\nPlease note: there is a second guest room at the Gold house, so if this listing is booked, do check with me to

Or try with another search query of "cozy room in austin":

In [9]:
mf.exec_query_s_2(input())

cozy room in austin


,title,description,city,url
Ranking,,,,
1,Entire Studio Apartment in West Campus Highrise,"Cozy studio apartment in high rise complex in west campus. There is ONE full sized bed (2 people) and room for an air mattress should YOU choose to bring one. More than 2 guests are welcome, but no additional bedding is provided. \nI can send a link to a video of the living area if you'd like. \n You are free to use the kitchen (pots, pans, etc) all you wish &amp; are welcome to store stuff in the fridge. \nTowels and sheets/blankets/pillows are provided. Feel free to reach out with any questions!",Austin,https://www.airbnb.com/rooms/19068268?location=Cedar%20Park%2C%20TX
2,"Peaceful, Private Condo in Southwest Austin","This is a peaceful, quiet place in Southwest Austin. It's surrounded by excellent restaurants and shopping, 15 minutes from South Congress and downtown. \n\nIt's the perfect place for dogs, with a big grassy area and miles of walking trails right behind the building. The pool area is always clean and quiet.\n\nThe inside is simple, clean and comfortable with great natural light. The living room is cozy with free Wifi and access to Xbox and Netflix. \n\n I'd love for you to stay here!",Austin,https://www.airbnb.com/rooms/18528747?location=Austin%2C%20TX
3,Two friends traveling the world,"This cozy private room, will make you feel at home during your journey around the world. \n\nInternational traveler friendly, private accommodation with the famous local charm will satisfy all your needs for a comfortable stay for two.\n\n15 min to downtown/riverwalk, 18 min to the airport and 1 hour to Austin, TX.",San Antonio,https://www.airbnb.com/rooms/17938496?location=Cibolo%2C%20TX
4,Two friends traveling the world,"This cozy private room, will make you feel at home during your journey around the world. \n\nInternational traveler friendly, private accommodation with the famous local charm will satisfy all your needs for a comfortable stay for two.\n\n15 min to downtown/riverwalk, 18 min to the airport and 1 hour to Austin, TX.",San Antonio,https://www.airbnb.com/rooms/17938496?location=Converse%2C%20TX
5,Cozy Private Room and Bath for SXSW,"This SXSW, enjoy the comfort of a home with a private bed &amp; bath, only 10 to 20 minutes (depending on traffic) from downtown. The neighborhood is quiet, safe, and has a park &amp; pool just a short walk away.",Austin,https://www.airbnb.com/rooms/17514280?location=Buda%2C%20TX
6,Comfy Guest Room - Harmony Place in South Austin,"Cozy Private Room in Harmony Place is close to everything! You can find yourself in Downtown Austin or Zilker Park where the city buzzes with Live Music and fabulous food in just 15 minutes. Public transportation is a short walk down the street. Harmony Place, named for it's Bohemian music vibe, is nestled on a quiet tree lined neighborhood. Lots of open spaces inside and out to relax and rejuvenate. Harmony Place is good for couples, solo adventurers, and business travelers.",Austin,https://www.airbnb.com/rooms/16543623?location=Buda%2C%20TX
7,Cozy Room with light Breakfast,"Hey, my place is 10 miles to Circuit of the Americas, 8 miles to the Airport. Very quiet community. The house has two bedrooms. Each bedroom has a queen size bed. Private bathroom. The house has high-ceiling throughout and large living area with all furnished. The kitchen is fully equipped with attached washer and dryer. I offer guests a light breakfast, such as coffee, juice, rolls, butter and jam. My place is good for couples, solo adventurers, business travelers, and families.",Austin,https://www.airbnb.com/rooms/15274066?location=Cedar%20Creek%2C%20TX
8,"Cozy, private loft with 2 bedrooms & family room","Our home is located in beautiful South Austin close to restaurants, parks, grocery stores, and shopping; its only 15-20 minutes from Zilker park and downtown! This Airbnb is ideal for couples, groups of up to 8, and guests who want a safe place to stay that is a short ride to Austin's nightlife

## Bonus Step: Make a nice visualization!

Our tool will works in the following way:<br/>

Take in input a set of coordinates, e.g: [32.779594, -96.800768], and a maximum distance from the coordinates, e.g.: 10km.<br/>
Generate a map, with a circle of the given radius, where the center is represented by the coordinates given in input.<br/>
Show the houses that are inside the circle of the given radius.<br/>

In [10]:
#Inputs
latitude, longitude = map(float,input().split())
max_distance = float(input()) #expressed in km
position = []
position.append(latitude)
position.append(longitude)

#We used: latitude = 32.779594, longitude = -96.800768, max_distance = 2

32.779594 -96.800768
2


In [11]:
#Representation of the User's position and area of interest
m = folium.Map(location=position,zoom_start=13.5) #the zoom_start depend on the area we want to represent

folium.Marker(
    location= position,
    icon=folium.Icon(color='green',icon='user')
    ).add_to(m)

folium.Circle(
    radius=max_distance*1000, #*1000 because radius needs a distance expressed in meters
    location=position,
    color='crimson',
    fill= True,
).add_to(m)

m

In the next map we will show all the places that are present within the given radius. Those are indicated with **red markers** are the ones with limited space left, while the **blue ones** imply many rooms are available at the place.<br/>
**Additionally, when user clicks on the icons it returns the number of rooms left of that AriBnB rental place**

In [12]:
#Calculation of distances and graphical representation
position2 = []
with open('Airbnb_Texas_Rentals.csv', 'r', encoding='utf-8' ) as csvfile:
    reader = csv.reader(csvfile)
    row1 = next(reader) # here we skip the first row of the .csv file
    for row in reader:
        if row[6]!='NA' and row[7]!='NA':
            position2.append(float(row[6]))
            position2.append(float(row[7]))
        if distance.distance(position, position2).km <= max_distance:
            #If there are few rooms (<=2) the icon is red, else the icon is blue
            if len(row[2])==1 and int(row[2])<=2:
                folium.Marker(location= position2,
                              popup = row[2],
                              icon=folium.Icon(color='red',icon='bed',prefix='fa')
                              ).add_to(m)
            else:
                 folium.Marker(location= position2, 
                          popup=row[2],
                          icon=folium.Icon(icon='bed',prefix='fa')
                          ).add_to(m)
        position2 = []
        
#Final representation. This one also shows the number of rooms of each AriBnB location when clicking on the marker
m